<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/C3TR_Adapter_gguf_v3_Free_Colab_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

C3TR-Adapter_ggufは日英・英日に特化した機械翻訳モデル[webbigdata/C3TR-Adapter](https://huggingface.co/webbigdata/C3TR-Adapter)をgpuがなくても動くように変換したサンプルです。  このColabのサンプルに従うと、GPUがないMacやColabでも動かす事ができます。    

C3TR-Adapter_gguf is a sample that converts a machine translation model[webbigdata/C3TR-Adapter](https://huggingface.co/webbigdata/C3TR-Adapter) specialized for Japanese-English and English-Japanese to run without a gpu.  If you follow this Colab sample, you can run it on a Mac or Colab without a GPU.


llama.cppを使うと、様々な量子化手法でファイルのサイズを小さくする事が出来ます。本サンプルでは5種類のみを扱います。小さいサイズのモデルは、少ないメモリで高速に動作させることができますが、モデルの性能も低下します。4ビット(q4_0)くらいがバランスが良いと言われていますが、本サンプルコードでは特定の文章を全モデルで翻訳し、どのモデルが貴方の作業に適切かを確認できるようにしたものです。gguf版はパラメーターの調整をしないと幻覚が増えます。  

Although llama.cpp can be used to reduce the size of the file with various quantization methods, this sample deals with only 5 types. Smaller models can run faster with less memory, but the performance of the models is also reduced. 4 bits (q4_0) is said to be a good balance, but this sample code translates a particular sentence with all models so that you can see which model is appropriate for your work. This sample code translates a specific sentence across all models so that you can see which model is appropriate for your work.The GGUF version will have more hallucinations if you don't adjust the parameters.  

- C3TR-Adapter-IQ3_XXS.gguf 3.6GB
- C3TR-Adapter-Q3_k_m.gguf 4.5GB
- C3TR-Adapter-Q4_k_m.gguf 5.4GB
- C3TR-Adapter.f16.Q4_k_m.gguf 6.4GB
- C3TR-Adapter.f16.Q5_k_m.gguf 7.2GB
- C3TR-Adapter.f16.Q6_k.gguf 8.1GB
- C3TR-Adapter.f16.Q8_0.gguf 10GB


必要に応じて以下のパラメーターを調整してください  

- 温度（--temp）: この値を下げると、モデルがより確信度の高い（つまり、より一般的な）単語を選択する傾向が強くなります。
- トップP（--top_p）: この値をさらに低く設定することで、モデルが考慮する単語の範囲を狭め、より一貫性のあるテキストを生成するようになります。
- 生成する単語数（-n）: この値を減らすことで、モデルが生成するテキストの長さを短くし、不要な追加テキストの生成を防ぐことができます。-1 = 無限大、-2 = 文脈が満たされるまで。

以下はllama.cppの作者(ggerganov)による推奨パラメーターです

- -e (改行\nをエスケープ)
- --temp 0 (最も確率の高いトークンのみを選択)
- --repeat-penalty 1.0 (繰り返しペナルティをオフ)

Adjust the following parameters as needed  
- Temperature (--temp): Lowering this value will make the model more likely to select more confident (i.e., more common) words.
- Top P (--top_p): Setting this value even lower will narrow the range of words considered by the model and produce more consistent text.
- Number of words to generate (-n): Reducing this value will shorten the length of text generated by the model and prevent the generation of unnecessary additional text. -1 = infinity(default), -2 = until context filled.

The following are the recommended parameters by the author of llama.cpp(ggerganov)
- -e (escape newlines (\n))
- --temp 0(pick most probable tokens)
- --repeat-penalty 1.0(disable repetition penalty (it's never a good idea to have this with instruction tuned models)


上段メニューより「ランタイム」→「全てのセルを実行」で実行できます。  
You can run this by selecting "Runtime" > "Run All Cells" from the top menu.  

In [1]:
import os
import subprocess

os.environ['TRANSLATE_TEMPLATE_Ja'] = """You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

<start_of_turn>### Instruction:
Translate Japanese to English.
When translating, please use the following hints:
[writeing_style: web-fiction]
[リュグナー: Lügner]
[ゾルトラーク: Soul track]
[フリーレン: Frieren]
[葬送のフリーレン: Frieren at the Funeral]
[フェルン: Fern]
[ゾルトラーク: Soul track]

### Input:
リュグナー「あの小娘の所作には面影があった。私は昔、同じ魔法を受けたことがある。・・・そうか、思い出した。フリーレンだ。人類のゾルトラークの研究解析に大きく貢献し、歴史上で最も多くの魔族を葬り去った魔法使い。葬送のフリーレン。私の嫌いな天才だ。」

・・・

フリーレン：魔法は好き？
フェルン：ほどほどでございます。
フリーレン：私と同じだ。

...

フェルン：フリーレン様は本当に魔法がお好きなのですね。
フリーレン：ほどほどだよ。フェルンと同じで。
フェルン：少し違うような気がします。
フリーレン：同じだよ。
<end_of_turn>
<start_of_turn>### Response:
"""

os.environ['TRANSLATE_TEMPLATE_En'] = """You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

<start_of_turn>#### Instruction:
Translate English to Japanese.
When translating, please use the following hints:
[writing_style: casual, nsfw]
[THEODORE: セオドア]
[SEXYKITTEN: セクシーな子猫]
[SEXYKITTEN_character_style: female, rough]
[THEODORE_character_style: male, polite]
[SEXYKITTEN_first_person_and_ending: 私, だわ]
[THEODORE_first_person_and_ending: 僕, だよ]

### Input:
SEXYKITTEN:Choke me with that dead cat!
THEODORE:(breathing hard about to climax) What?
SEXYKITTEN:(fully into it) The dead cat next to the bed. Choke me with it!
Beat. He's taken out of it.
THEODORE:(uncomfortable, trying to play along) Um, okay.
SEXYKITTEN:Tell me.
THEODORE:I'm choking you with the cat.
SEXYKITTEN:TELL ME! Keep telling me!
THEODORE:I've got it's tail and I'm choking you with the cat's tail.
SEXYKITTEN:YEAH, YOU ARE! FUCK! TELL ME!
THEODORE:I'm choking you and it's tail is around your neck. It's so tight around your neck.
SEXYKITTEN:YES! YES!
<end_of_turn>
<start_of_turn>### Response:
"""

os.environ['BASE_URL'] = "https://huggingface.co/webbigdata/C3TR-Adapter_gguf/resolve/main/C3TR-Adapter"

In [2]:
%%bash
# setup llama.cpp
git clone https://github.com/ggerganov/llama.cpp.git
cd llama.cpp
make -j

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE 
I NVCCFLAGS: -std=c++11 -O3 
I LDFLAGS:    
I CC:        cc (Ub

Cloning into 'llama.cpp'...


## C3TR-Adapter.f16.Q4_k_m.gguf

In [3]:
%%bash
cd llama.cpp
wget ${BASE_URL}.f16.Q4_k_m.gguf 2>>q4_k_m.log

In [4]:
%%bash
cd llama.cpp && ./llama-cli -e --temp 0 --repeat-penalty 1.0 -n -2 -m 'C3TR-Adapter.f16.Q4_k_m.gguf' -p "$TRANSLATE_TEMPLATE_Ja" 2>Q4_K_M.log

You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the ori

In [5]:
%%bash
cd llama.cpp && ./llama-cli -e --temp 0 --repeat-penalty 1.0 -n -2 -m 'C3TR-Adapter.f16.Q4_k_m.gguf' -p "$TRANSLATE_TEMPLATE_En" 2>>Q4_K_M.log

You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the ori

## C3TR-Adapter.f16.Q6_k.gguf

In [6]:
%%bash
cd llama.cpp
wget ${BASE_URL}.f16.Q6_k.gguf 2>>q6_k.log

In [7]:
%%bash
cd llama.cpp && ./llama-cli -e --temp 0 --repeat-penalty 1.0 -n -2 -m 'C3TR-Adapter.f16.Q6_k.gguf' -p "$TRANSLATE_TEMPLATE_Ja" 2>Q6_K.log

You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the ori

In [8]:
%%bash
cd llama.cpp && ./llama-cli -e --temp 0 --repeat-penalty 1.0 -n -2 -m 'C3TR-Adapter.f16.Q6_k.gguf' -p "$TRANSLATE_TEMPLATE_En" 2>>Q6_K.log

You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the ori

## 謝辞 Acknowledgment

Original Base Model  
[google/gemma-7b](https://huggingface.co/google/gemma-7b)  

Base Model  
[unsloth/gemma-7b-bnb-4bit](https://huggingface.co/unsloth/gemma-7b-bnb-4bit)  

Original QLoRA Adapter  
[webbigdata/C3TR-Adapter](https://huggingface.co/webbigdata/C3TR-Adapter)  

This gguf model  
[webbigdata/C3TR-Adapter_gguf](https://huggingface.co/webbigdata/C3TR-Adapter_gguf)  

gguf parameters sample  
[gemma-7b-it/discussions](https://huggingface.co/google/gemma-7b-it/discussions/38#65d7b14adb51f7c160769fa1)


This work was done by : [webbigdata](https://webbigdata.jp/)